In [1]:
from neo4j import GraphDatabase
import json
from random import random
class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.driver = GraphDatabase.driver(uri, auth=(user, pwd))

    def close(self):
        self.driver.close()

    def query(self, query, parameters=None, db=None):
        with self.driver.session(database=db) as session:
            result = session.run(query, parameters)
            # Collect all records at once to avoid ResultConsumedError
            return [record for record in result]

def fetch_and_format_data(uri, user, pwd):
    # Initialize Neo4j connection
    conn = Neo4jConnection(uri=uri, user=user, pwd=pwd)

    # Define the Cypher query to fetch the relevant data
    query = """
    MATCH (theme:Theme {name: "EPU_POLICY"})<-[:RELATED_TO]-(news:News)
    OPTIONAL MATCH (theme)-[:ASSOCIATED_WITH]->(o:Organization)
    OPTIONAL MATCH (theme)-[:INVOLVES]->(person:Person)
    WHERE news.title CONTAINS 'asdasdasd'
    RETURN theme.name as relatedEntity,
      news.title as newsTitle,
      person.name AS personName,
        o.name as orgName
    LIMIT 20
    """

    # Execute the query and collect all results
    results = conn.query(query)
    print(results)

    # Structure the data into the desired JSON format
    output_data = {
        "name": "EPU_POLICY",
        "children": [
            {"name": "News", "children": []},
            {"name": "Persons", "children": []},
            {"name": "Organizations", "children": []}
        ]
    }

    # Process each record and add to the appropriate category
    for record in results:
        # Add to News
        if record["newsTitle"]:
            output_data["children"][0]["children"].append({
                "id":f"new_{random()}",
                "name": record["newsTitle"],
                #"topic": record["relatedEntity"],
                #"link": f"http://example.com/news/{record['newsTitle'].replace(' ', '_')}",
                "size": 400
            })

        # Add to Persons
        if record["personName"]:
            output_data["children"][1]["children"].append({
                "id":f"person_{random()}",
                "name": record["personName"],
                #"topic": record["relatedEntity"],
                "size": 400
            })

        # Add to Organizations
        if record["orgName"]:
            output_data["children"][2]["children"].append({
                "id":f"org_{random()}",
                "name": record["orgName"],
                #"topic": record["relatedEntity"],
                "size": 400
            })

    # Close the Neo4j connection
    conn.close()

    # Export the data to a JSON file
    with open('output_graph.json', 'w') as outfile:
        json.dump(output_data, outfile, indent=2)

    print("Data exported to output_graph.json")
if __name__ == "__main__":
    # Connection details
    uri = "neo4j+s://7069a022.databases.neo4j.io"  # Update with your Neo4j instance URI
    user = "neo4j"                  # Update with your Neo4j username
    pwd = "qWWshGvcU5a-bSOCgOj30ViKHW5ows0vtFMjWZh3djg"                # Update with your Neo4j password

    # Fetch data and generate JSON
    fetch_and_format_data(uri, user, pwd)


ValueError: Cannot resolve address 7069a022.databases.neo4j.io:7687

In [4]:
from neo4j import GraphDatabase
import json
from random import random
class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.driver = GraphDatabase.driver(uri, auth=(user, pwd))

    def close(self):
        self.driver.close()

    def query(self, query, parameters=None, db=None):
        with self.driver.session(database=db) as session:
            result = session.run(query, parameters)
            # Collect all records at once to avoid ResultConsumedError
            return [record for record in result]

In [56]:
query = """MATCH (e:News)-[:HAS_THEME]->(t:Theme)
OPTIONAL MATCH (e)-[:INVOLVES]->(p:Person)
OPTIONAL MATCH (e)-[:INVOLVES_ORGANIZATION]->(o:Organization)
OPTIONAL MATCH (e)-[:REPORTED_BY]->(s:Source)
WHERE e.date = '13-08-2024'
RETURN e.id AS NewsID, 
       e.title AS Title, 
       e.date AS Date, 
       e.tone AS Tone, 
       e.content AS Content, 
       e.first_sourceurl AS SourceURL,
       t.name AS Topic, 
       p.name AS Person, 
       o.name AS Organization, 
       s.name AS Source,
      'HAS_THEME' AS RelacionTopic,
       'INVOLVES' AS RelacionPersona, 
       'INVOLVES_ORGANIZATION' AS RelacionOrganizacion
ORDER BY e.id ASC, t.name ASC, p.name ASC, o.name ASC
LIMIT 100
"""

conn = Neo4jConnection(uri="neo4j+s://614dfd28.databases.neo4j.io", user="neo4j" , pwd="iSqda4HLJiAZo1TgAyRH0SrvYyj9Nd3olNj6PvZ33sE")

results = conn.query(query)
print(results)


[<Record NewsID='10235' Title='Court Ruling Allows Village To Regulate Camping' Date='13-08-2024' Tone='0.946372239747634,4.10094637223975,3.15457413249211,7.25552050473186,23.6593059936909,0' Content='A recent ruling by the U.S. Supreme Court that reversed a lower court’s decision that allowed the imposition of penalties against unauthorized camping led to approval of a local ordinance regulating the practice.\n\nRosemont Village Board members approved the ordinance that allows regulation of camping whenever it arises. Very few instances have been recorded where people have camped outdoors in Rosemont. The only comparison in recent years is that some concert-goers have formed long lines in front of the Allstate Arena to purchase tickets and buy band merchandise. There has also been some outdoor camping with tents for these concerts, but they are restricted to property across from the arena on Lunt Avenue.\n\nLast weekend, the K-pop band Ateez performed two shows at the arena to sold-o

In [57]:
data_final =[]

for result in results:
    data_final.append(dict(result))

In [58]:
data_final

# Guardar la estructura JSON en un archivo
output_file = "output_json_final_final.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data_final, f, indent=4, ensure_ascii=False)

print(f"JSON guardado en el archivo: {output_file}")

JSON guardado en el archivo: output_json_final_final.json


In [62]:
import json
from collections import defaultdict

# Cargar los datos desde el archivo JSON
input_file = "output_json_final_final.json"
with open(input_file, 'r', encoding='utf-8') as f:
    data_final = json.load(f)

# Crear la estructura jerárquica
hierarchy = defaultdict(lambda: defaultdict(dict))

for entry in data_final:
    fecha = entry['Date']
    noticia = entry['Title']
    
    # Separar las relaciones en diccionarios distintos
    persona_dict = {
        "Persona": entry['Person'],
        "RelacionPersona": entry['RelacionPersona']
    }
    
    organizacion_dict = {
        "Organizacion": entry['Organization'],
        "RelacionOrganizacion": entry['RelacionOrganizacion']
    }
    
    topic_dict = {
        "Topic": entry['Topic'],
        "RelacionTopic": entry['RelacionTopic']
    }
    
    # Añadir los diccionarios separados a la jerarquía
    hierarchy[fecha][noticia]['Relaciones'] = {
        "Persona": persona_dict,
        "Organizacion": organizacion_dict,
        "Topico": topic_dict
    }
    
    # Añadir embeddings, título y fuente a la noticia
    hierarchy[fecha][noticia]['embedding'] = entry.get('Tone')
    hierarchy[fecha][noticia]['title'] = entry.get('Title')
    hierarchy[fecha][noticia]['source'] = entry.get('SourceURL')

# Convertir la estructura a una lista para serializar como JSON
hierarchy_json = []

for fecha, noticias in hierarchy.items():
    fecha_dict = {"Fecha": fecha, "Noticias": []}
    for noticia, detalles in noticias.items():
        noticia_dict = {
            "Noticia": noticia,
            "embedding": detalles.get('embedding'),
            "title": detalles.get('title'),
            "source": detalles.get('source'),
            "Relaciones": detalles['Relaciones']
        }
        fecha_dict["Noticias"].append(noticia_dict)
    hierarchy_json.append(fecha_dict)

# Convertir a JSON
json_output = json.dumps(hierarchy_json, indent=4, ensure_ascii=False)
print(json_output)

# Guardar la estructura JSON en un archivo
output_file = "output_json_separated_relationships.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(hierarchy_json, f, indent=4, ensure_ascii=False)

print(f"JSON guardado en el archivo: {output_file}")



[
    {
        "Fecha": "13-08-2024",
        "Noticias": [
            {
                "Noticia": "Court Ruling Allows Village To Regulate Camping",
                "embedding": "0.946372239747634,4.10094637223975,3.15457413249211,7.25552050473186,23.6593059936909,0",
                "title": "Court Ruling Allows Village To Regulate Camping",
                "source": "https://www.journal-topics.com/articles/court-ruling-allows-village-to-regulate-camping",
                "Relaciones": {
                    "Persona": {
                        "Persona": null,
                        "RelacionPersona": "INVOLVES"
                    },
                    "Organizacion": {
                        "Organizacion": "u s supreme court",
                        "RelacionOrganizacion": "INVOLVES_ORGANIZATION"
                    },
                    "Topico": {
                        "Topic": "USPEC_POLICY1",
                        "RelacionTopic": "HAS_THEME"
                    }


In [20]:
output_data = {
"name": "EPU_CATS_TAXES",
"children": [
    {"name": "News","relation":"", "children": []},
    {"name": "Persons","relation":"", "children": []},
    {"name": "Organizations","relation":"", "children": []}
]
}

# Process each record and add to the appropriate category
for record in results:

    # Add to News
    if record["Noticia"]:
        output_data["children"][0]["relation"]=record["RelNews"]
        output_data["children"][0]["children"].append({
            "id":f"new_{random()}",
            "name": record["Noticia"],
            "img":"http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_spiderman.png",
            "size": 400
        })

    # Add to Persons
    if record["Persona"]:
        output_data["children"][1]["relation"]=record["RelPerson"]
        output_data["children"][1]["children"].append({
            "id":f"person_{random()}",
            "name": record["Persona"],
            "img":"http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_spiderman.png",
            #"topic": record["relatedEntity"],
            "size": 400
        })

    # Add to Organizations
    if record["Org"]:
        output_data["children"][2]["relation"]=record["RelOrg"]
        output_data["children"][2]["children"].append({
            "id":f"org_{random()}",
            "name": record["Org"],
            #"topic": record["relatedEntity"],
            "img":"http://marvel-force-chart.surge.sh/marvel_force_chart_img/top_spiderman.png",
            "size": 400
        })

In [21]:
with open('output_graph_new.json', 'w') as outfile:
        json.dump(output_data, outfile, indent=2)

In [43]:
import json
from collections import defaultdict

# Datos simulados basados en el archivo JSON proporcionado

# Crear la estructura jerárquica
hierarchy = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for entry in data_final:
    fecha = entry['Fecha']
    noticia = entry['Noticia']
    
    # Añadir las relaciones con embeddings a la jerarquía
    hierarchy[fecha][noticia]['Relaciones'].append({
        "Persona": entry['Persona'],
        "RelacionPersona": entry['RelPerson'],
        "Organizacion": entry['Org'],
        "RelacionOrganizacion": entry['RelOrg']
    })
    
    # Añadir embeddings a la noticia
    hierarchy[fecha][noticia]['embedding'] = entry['embedding']
    hierarchy[fecha][noticia]['title'] = entry['title']

# Convertir la estructura a una lista para serializar como JSON
hierarchy_json = []

for fecha, noticias in hierarchy.items():
    fecha_dict = {"Fecha": fecha, "Noticias": []}
    for noticia, detalles in noticias.items():
        noticia_dict = {
            "Noticia": noticia,
            "embedding": detalles['embedding'],
            "title":detalles["title"],
            "Relaciones": detalles['Relaciones']
        }
        fecha_dict["Noticias"].append(noticia_dict)
    hierarchy_json.append(fecha_dict)

# Convertir a JSON
json_output = json.dumps(hierarchy_json, indent=4, ensure_ascii=False)
print(json_output)

# Guardar la estructura JSON en un archivo
output_file = "output_json_final.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(hierarchy_json, f, indent=4, ensure_ascii=False)

print(f"JSON guardado en el archivo: {output_file}")

[
    {
        "Fecha": "13-08-2024",
        "Noticias": [
            {
                "Noticia": "(NEW YORK) — Junk, resort, destination, urban, and amenity fees are pseudonyms for the mandatory, often unexpected surcharges you might find tackled on your hotel bill. According to the Council of Economic Advisers (CEA), these fees cost Americans nearly $3.4 billion annually and despite recent bipartisan efforts by the Biden administration to combat junk fees entirely, they still seem to pop up when it’s time to pay for your stay.\n\n“The consumer is the loser” when it comes to junk fees, Clint Henderson, managing editor at The Points Guy, told ABC News’ Good Morning America.\n\nThe value of what’s being delivered has fallen dramatically for consumers, who are now paying all-encompassing fees for erroneous line items such as Wi-Fi, phone calls, fax machines, towels, beach access, breakfast, parking, fitness centers and more — the list goes on, and “it’s getting hard to keep up,” Hend